# Проект: Прогнозирование оттока клиентов (Churn Prediction)

## Контекст проекта

Компания стремится снизить отток клиентов и повысить удержание.  
Для этого необходимо заранее выявлять клиентов с высокой вероятностью ухода.

В рамках проекта разрабатывается модель бинарной классификации, 
которая прогнозирует вероятность оттока на основе поведенческих 
и транзакционных данных пользователей.


## Цель проекта

Построить и оценить модель машинного обучения, способную:

- прогнозировать вероятность оттока клиента,
- корректно работать при дисбалансе классов,
- выдавать калиброванные вероятности,
- быть готовой к интеграции в продакшен (через сохранённый Pipeline).

# Модель для прогнозирования оттока клиентов для сервиса доставки кофе

# План работы

## Этап 1. Подготовка среды и библиотек
1. Установите и настройте библиотеки. Для воспроизводимости результатов зафиксируйте версии пакетов в файле `requirements.txt`.

2. Зафиксируйте `random_state`.

3. Загрузите данные из CSV-файла. Путь к файлу: `'/datasets/coffee_churn_dataset.csv'`. Используйте сепаратор `","`, а для чтения чисел с плавающей точкой — параметр `decimal="."`.

In [ ]:
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import average_precision_score

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn, matplotlib

print(
    "pandas:", pd.__version__,
    "| numpy:", np.__version__,
    "| sklearn:", sklearn.__version__,
    "| matplotlib:", matplotlib.__version__,
    "| seaborn:", sns.__version__,
    "| joblib:", joblib.__version__
)

In [ ]:
requirements = """
pandas==1.2.4
numpy==1.21.1
scikit-learn==0.24.1
matplotlib==3.3.4
seaborn==0.11.1
joblib==1.1.0
"""

with open("requirements.txt", "w") as f:
    f.write(requirements)

Для воспроизводимости результатов зафиксированы версии используемых библиотек в файле requirements.txt.

In [ ]:
RANDOM_STATE = 42

Для обеспечения воспроизводимости экспериментов зафиксирован параметр random_state.

In [ ]:
df = pd.read_csv(
    '/datasets/coffee_churn_dataset.csv',
    sep=',',
    decimal='.'
)

df.head()

In [ ]:
df.shape

Данные успешно загружены.<br>
Датасет содержит 10450 наблюдений и 27 признаков.<br>
В выборке присутствуют числовые и категориальные признаки, а также целевая переменная churn, отражающая факт оттока клиента.<br>

## Этап 2. Первичный анализ данных

1. Опишите данные. Кратко сообщите, что известно о пользователях и их поведении.

2. Опишите целевую переменную. Обратите внимание на возможные особенности её распределения. Проверьте, наблюдается ли дисбаланс классов в целевой переменной.

3. Опишите признаки.

   - Определите, все ли из них важны.

   - Объясните, какие из них можно удалить (если такие есть). Аргументируйте своё решение.

4. Обработайте пропущенные значения.
   
   - Объясните, как они влияют на данные.

   - Выберите стратегию заполнения пропусков.

5. Проанализируйте категориальные признаки.

   - Выясните, есть ли в данных признаки, которые можно кодировать. Объясните, почему именно их нужно кодировать.

   - Проанализируйте признаки на предмет того, можно ли использовать некоторые из них для генерации новых  признаков. Укажите возможные стратегии.

   - Определите, есть ли в данных признаки, которые можно удалить.

6. Проанализируйте выбросы.

   - Определите, как они влияют на данные.

   - Выберите способ, которым их можно обработать.

7. Посчитайте корреляции между признаками. Постройте необходимые визуализации. Определите, есть ли признаки, которые можно убрать, на основании их корреляции с другими.

8. Напишите выводы по результатам исследовательского анализа данных.

In [ ]:
df['churn'].value_counts()

In [ ]:
df['churn'].value_counts(normalize=True)

Целевая переменная churn отражает факт ухода клиента.<br>
В выборке наблюдается выраженный дисбаланс классов: доля ушедших клиентов составляет около 6%.<br>

В связи с сильным дисбалансом использование метрики Accuracy некорректно, так как модель может демонстрировать высокую точность, просто предсказывая большинство класса.<br>

В качестве основной метрики выбрана PR-AUC, поскольку она лучше отражает качество модели при работе с редким положительным классом.<br>

In [ ]:
df.info()

Признак user_id является уникальным идентификатором пользователя и не несёт прогностической ценности.<br>
Он будет удалён перед обучением модели, чтобы избежать утечки информации и переобучения.

In [ ]:
df = df.drop(columns=['user_id'])

In [ ]:
missing = df.isna().mean().sort_values(ascending=False)
missing

В данных присутствуют пропуски почти во всех признаках, но их доля относительно небольшая (в основном до 10%).<br>
Удаление строк с пропусками приведёт к потере значимой части данных, поэтому пропуски будут обрабатываться с помощью импутации.

Для числовых признаков будет использована замена на медиану (устойчива к выбросам), для категориальных замена на наиболее частое значение.<br>
Импутация будет выполнена внутри Pipeline, чтобы избежать утечки данных при кросс-валидации.

In [ ]:
cat_cols = df.select_dtypes(include='object').columns
cat_cols

In [ ]:
df[cat_cols].nunique().sort_values(ascending=False)

Все категориальные признаки будут закодированы с помощью One-Hot Encoding.<br>
Признак geo_location содержит 100 уникальных значений, однако размер датасета позволяет использовать его без удаления.

Для предотвращения ошибок при появлении новых категорий будет использован параметр handle_unknown='ignore'.

In [ ]:
bin_cols = ['seasonal_menu_tried', 'notifications_enabled', 'coffee_preference_change']

for col in bin_cols:
    print(col, df[col].unique())

Признаки seasonal_menu_tried, notifications_enabled и coffee_preference_change являются бинарными (0/1).<br>
Пропущенные значения в них будут заполнены наиболее частым значением.<br>
Дополнительного кодирования для них не требуется.<br>

In [ ]:
num_cols = df.select_dtypes(include=['float64']).columns
num_cols

В датасете присутствуют 16 числовых признаков, отражающих поведение пользователей: частоту заказов, средний чек, расходы, активность в приложении, использование скидок и другие характеристики.

Числовые признаки будут:

- заполнены медианой (для устойчивости к выбросам)

- стандартизированы перед обучением модели.

In [ ]:
df[num_cols].describe().T

В числовых признаках обнаружены выбросы и аномальные значения.<br>
В частности, в ряде финансовых признаков присутствуют отрицательные значения, что экономически невозможно и может быть связано с особенностями генерации данных.

Также наблюдаются сильные правые хвосты распределений (например, total_spent_last_month, app_opens_per_week).

Поскольку доля таких значений невелика, удаление наблюдений может привести к потере информации.<br>
Для минимизации влияния выбросов будет использована медианная импутация и стандартизация признаков в рамках Pipeline.

In [ ]:
financial_cols = [
    'avg_order_value',
    'median_order_value',
    'total_spent_last_month',
    'total_spent_last_week'
]

for col in financial_cols:
    df = df[df[col] >= 0]

In [ ]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = Q1 - 3 * IQR
    upper = Q3 + 3 * IQR
    
    df = df[(df[col] >= lower) & (df[col] <= upper)]

In [ ]:
print("Размер данных после очистки:", df.shape)

На основании анализа описательной статистики выявлены экономически невозможные и экстремальные значения.

Перед построением модели произведена предварительная очистка данных.

In [ ]:
df[['notifications_enabled', 'coffee_preference_change']].nunique()

In [ ]:
df = df.drop(columns=['notifications_enabled', 'coffee_preference_change'])

После предварительной очистки признаки `notifications_enabled` и `coffee_preference_change` стали константными (nunique = 1).<br>
Поскольку такие признаки не несут информации и не влияют на модель, они были исключены из дальнейшего анализа.

In [ ]:
plt.figure(figsize=(14, 12))

num_cols = df.select_dtypes(include='number').columns.tolist()
corr_matrix = df[num_cols].corr()

sns.heatmap(
    corr_matrix,
    cmap='coolwarm',
    center=0,
    annot=True,
    fmt=".2f",
    square=True,
    linewidths=0.5
)

plt.title("Correlation Matrix", fontsize=16)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
corr_matrix = df[num_cols].corr().abs()

upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

high_corr = (
    upper_triangle
    .stack()
    .sort_values(ascending=False)
)

high_corr[high_corr > 0.7]

При анализе корреляций выявлена крайне высокая зависимость (0.99) между признаками order_frequency_month и order_frequency_week, что свидетельствует о практически линейной связи между ними.

Для снижения мультиколлинеарности и повышения интерпретируемости модели признак order_frequency_week был удалён.

Остальные признаки демонстрируют умеренную корреляцию, которая не является критичной при использовании регуляризованной логистической регрессии.

In [ ]:
df = df.drop(columns=['order_frequency_week'])

Признаки avg_order_value и median_order_value демонстрируют высокую корреляцию (0.87), однако они отражают разные аспекты распределения расходов клиента.<br>
Среднее чувствительно к выбросам, тогда как медиана более устойчива.<br>
В связи с этим оба признака сохраняются в модели.

В ходе анализа данных выявлен выраженный дисбаланс классов (доля оттока около 6%), что обосновывает использование метрики PR-AUC.<br>
В данных присутствуют пропуски и выбросы, поэтому выбрана стратегия медианной импутации и стандартизации признаков в рамках Pipeline.<br>
Признак user_id удалён как неинформативный идентификатор.<br>
Также удалён признак order_frequency_week из-за высокой корреляции с order_frequency_month.<br>
Данные подготовлены к построению модели.

Построим распределения числовых признаков, чтобы увидеть выбросы, правые хвосты и аномальные значения.

In [ ]:
num_cols = df.select_dtypes(include='number').columns.tolist()

for col in num_cols:
    plt.figure(figsize=(8, 4))
    plt.hist(df[col].dropna(), bins=40)
    plt.title(f"Distribution: {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

Анализ гистограмм показал, что большинство поведенческих и финансовых признаков демонстрируют выраженный правый хвост распределения.

Это соответствует природе данных (количество событий за период времени),что близко к пуассоновскому распределению.

Для признаков:

- `order_frequency_month`
- `total_spent_last_month`
- `total_spent_last_week`
- `app_opens_per_week`
- `days_since_last_promo`

наблюдается значительное влияние экстремально высоких значений.

Поскольку линейная модель чувствительна к масштабу признаков, для уменьшения влияния высоких значений и моделирования нелинейной зависимости будет применено логарифмирование (функция log1p) на этапе feature engineering.

Это позволит:
- усилить различия в области малых значений,
- сгладить влияние правого хвоста,
- повысить устойчивость модели.

## Этап 3. Предобработка данных

1. Разделите данные в пропорции 80 к 20. 20% данных отложите для теста. Остальные используйте для обучения и кросс-валидации модели.

2. Предобработайте данные. Используйте информацию о пропусках и категориальных признаках только из обучающей выборки.

   - Создайте пайплайн, который обработает пропуски и выбросы.

   - Создайте пайплайн, который обработает категориальные признаки.

   - Создайте пайплайн, который обработает числовые признаки: проведёт масштабирование и нормализацию.



In [ ]:
X = df.drop(columns=['churn'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Данные разделены в пропорции 80/20 с использованием стратификации по целевой переменной для сохранения распределения классов.<br>
Тестовая выборка отложена и не будет использоваться при обучении и кросс-валидации.

In [ ]:
num_cols = X_train.select_dtypes(include=['float64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

print("Numerical:", len(num_cols))
print("Categorical:", len(cat_cols))

Типы признаков определены на обучающей выборке, чтобы избежать утечки данных.<br>
Выделено 13 числовых и 9 категориальных признаков.

In [ ]:
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [ ]:
cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

Предобработка реализована через Pipeline и ColumnTransformer, чтобы все трансформации выполнялись только на обучающих данных внутри кросс-валидации и не возникало утечки данных.

Для числовых признаков используется медианная импутация и стандартизация (StandardScaler), что снижает влияние выбросов и приводит признаки к сопоставимому масштабу.
Для категориальных признаков используется заполнение наиболее частым значением и One-Hot Encoding с handle_unknown='ignore'.

## Этап 4. Обучение модели

1. Обучите базовую версию модели.
   - Используйте для этого простые статистические модели.

   - Используйте кросс-валидацию для обучения модели.

2. Посчитайте метрики, поставленные в задаче. Опираясь на них, сделайте вывод о качестве модели.

In [ ]:
baseline_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

baseline_cv = cross_validate(
    baseline_model,
    X_train,
    y_train,
    cv=cv,
    scoring="average_precision",
    n_jobs=-1,
    return_train_score=False
)

print("Baseline PR-AUC (mean):", baseline_cv["test_score"].mean())
print("Baseline PR-AUC (std):", baseline_cv["test_score"].std())
print("Scores:", baseline_cv["test_score"])

Базовая модель (DummyClassifier, стратегия most_frequent) показала PR-AUC 0.06.<br>
Это значение совпадает с долей положительного класса в данных 6%, что соответствует ожидаемому уровню случайного классификатора при использовании PR-AUC.

Таким образом, полученный результат служит нижней границей качества.
Для практического применения модель должна демонстрировать PR-AUC существенно выше этого уровня.

In [ ]:
logreg_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(
        random_state=RANDOM_STATE,
        max_iter=1000
    ))
])

logreg_cv = cross_validate(
    logreg_model,
    X_train,
    y_train,
    cv=cv,
    scoring="average_precision",
    n_jobs=-1,
    return_train_score=False
)

print("LogReg PR-AUC (mean):", logreg_cv["test_score"].mean())
print("LogReg PR-AUC (std):", logreg_cv["test_score"].std())
print("Scores:", logreg_cv["test_score"])

Логистическая регрессия показала среднее значение PR-AUC 0.656 по результатам 5-кратной стратифицированной кросс-валидации.

Это существенно превышает baseline (0.06), что свидетельствует о способности модели выявлять уходящих клиентов.<br>
Разброс метрики между фолдами умеренный (std = 0.05), что указывает на стабильность модели.

## Этап 5. Создание новых признаков

1. Добавьте новые признаки, которые могут улучшить качество модели. Опирайтесь на наработки, полученные в ходе исследовательского анализа данных, и на логику решаемой задачи.

   - Извлечение квадратного корня поможет сгладить большие значения.

   - Возведение в квадрат усилит влияние больших значений.

2. Обновите пайплайн для работы с новыми признаками, проведите повторную кросс-валидацию, сравните результаты моделей с новыми признаками и без них.

3. Интерпретируйте коэффициенты модели, а затем на их основании выявите значимые признаки и удалите лишние для модели.

In [ ]:
def add_features(X):
    X = X.copy()

    def safe_log1p(s):
        return np.log1p(np.clip(s, 0, None))

    def safe_sqrt(s):
        return np.sqrt(np.clip(s, 0, None))

    if "total_spent_last_week" in X.columns:
        X["log_total_spent_last_week"] = safe_log1p(X["total_spent_last_week"])

    if "total_spent_last_month" in X.columns:
        X["log_total_spent_last_month"] = safe_log1p(X["total_spent_last_month"])

    if "avg_order_value" in X.columns:
        X["log_avg_order_value"] = safe_log1p(X["avg_order_value"])

    if "order_frequency_month" in X.columns:
        X["order_frequency_month_sq"] = X["order_frequency_month"] ** 2
        X["log_order_frequency_month"] = safe_log1p(X["order_frequency_month"])

    if "app_opens_per_week" in X.columns:
        X["sqrt_app_opens_per_week"] = safe_sqrt(X["app_opens_per_week"])
        X["log_app_opens_per_week"] = safe_log1p(X["app_opens_per_week"])

    if "days_since_last_promo" in X.columns:
        X["log_days_since_last_promo"] = safe_log1p(X["days_since_last_promo"])

    return X


feature_engineering = FunctionTransformer(add_features)

По результатам исследовательского анализа:

Финансовые и поведенческие признаки (total_spent_last_month, total_spent_last_week, order_frequency_month, app_opens_per_week,
days_since_last_promo) имеют выраженную правостороннюю асимметрию и выбросы. Это видно по гистограммам: длинный правый хвост и концентрация наблюдений в области малых значений.

Применяется log1p, чтобы:

- сгладить влияние экстремальных значений,
- приблизить зависимость к линейной
- снизить чувствительность логистической регрессии к крупным значениям.

order_frequency_month

По EDA видно, что с ростом частоты заказов вероятность оттока снижается нелинейно: эффект усиливается при больших значениях.

Добавляется квадрат признака (order_frequency_month_sq),
чтобы позволить линейной модели учитывать нелинейный эффект усиления.

app_opens_per_week

Распределение имеет выраженный хвост, но эффект насыщается при росте значения.

Добавляется sqrt_app_opens_per_week,
чтобы смоделировать эффект убывающей отдачи (diminishing returns).

In [ ]:
X_tmp = add_features(X_train)

print("shape:", X_tmp.shape)
print("NaN count:", X_tmp.isna().sum().sum())
print("Inf count:", np.isinf(X_tmp.select_dtypes(include=[np.number])).sum().sum())

cols_to_show = [
    "total_spent_last_month",
    "total_spent_last_week",
    "log_total_spent_last_week",
    "avg_order_value",
    "log_avg_order_value",
    "app_opens_per_week",
    "sqrt_app_opens_per_week",
    "order_frequency_month",
    "order_frequency_month_sq",
]

cols_to_show = [c for c in cols_to_show if c in X_tmp.columns]
X_tmp[cols_to_show].head()

In [ ]:
print("NaN в оригинальном X_train:",
      X_train.isna().sum().sum())

print("NaN после add_features:",
      X_tmp.isna().sum().sum())

В рамках feature engineering были добавлены нелинейные признаки: логарифмы финансовых показателей, квадрат частоты заказов и квадратный корень активности в приложении.<br>
Для предотвращения вычислительных ошибок использована безопасная трансформация (clipping отрицательных значений).<br>
Пропущенные значения сохраняются и будут обработаны в пайплайне на этапе импутации.

<div class="alert alert-info">

<h2>Комментарий студента<a class="tocSkip"></a></h2>

<b>Доработано:</b><br/>
В рамках feature engineering добавлены нелинейные признаки: логарифмы финансовых показателей, квадрат частоты заказов и квадратный корень активности в приложении.

Выбор преобразований основан на результатах EDA:
финансовые признаки имеют выраженную правую асимметрию — логарифмирование сглаживает распределение и снижает влияние экстремальных значений;
для order_frequency_month наблюдается усиление эффекта при больших значениях — добавлен квадрат признака;
для app_opens_per_week предполагается эффект убывающей отдачи — применён квадратный корень.

Поскольку используется логистическая регрессия (линейная модель), такие преобразования позволяют учесть нелинейные зависимости без смены алгоритма и потенциально улучшить качество модели.

</div>

In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor_fe = ColumnTransformer(
    transformers=[
        ("num", num_pipeline, selector(dtype_include=np.number)),
        ("cat", cat_pipeline, selector(dtype_exclude=np.number)),
    ]
)

pipe_fe = Pipeline([
    ("feature_engineering", feature_engineering),
    ("preprocessing", preprocessor_fe),
    ("model", LogisticRegression(max_iter=1000, random_state=42))
])

In [ ]:
X_reduced = X.drop(columns=["avg_order_value"])

fe_scores = cross_val_score(
    pipe_fe,
    X,
    y,
    cv=cv,
    scoring="average_precision"
)

print("FE PR-AUC (mean):", fe_scores.mean())
print("FE PR-AUC (std):", fe_scores.std())
print("Scores:", fe_scores)

После добавления нелинейных признаков (логарифмические преобразования финансовых показателей, квадрат частоты заказов и квадратный корень активности в приложении) качество модели улучшилось.<br>
Среднее значение PR-AUC увеличилось с 0.656 до 0.714, а разброс между фолдами снизился, что свидетельствует о повышении стабильности модели.<br>
Таким образом, feature engineering положительно повлиял на способность модели выявлять уходящих клиентов.

In [ ]:
pipe_fe.fit(X_train, y_train)

preprocessor = pipe_fe.named_steps["preprocessing"]

num_cols_used = list(preprocessor.transformers_[0][2])
cat_cols_used = list(preprocessor.transformers_[1][2])

ohe = preprocessor.named_transformers_["cat"].named_steps["onehot"]
cat_feature_names = ohe.get_feature_names(cat_cols_used)

feature_names = np.concatenate([num_cols_used, cat_feature_names])
coefs = pipe_fe.named_steps["model"].coef_.ravel()

print("n_features:", len(feature_names), "n_coefs:", len(coefs))

coef_df = (pd.DataFrame({
    "feature": feature_names,
    "coef": coefs,
    "abs_coef": np.abs(coefs)
}).sort_values("abs_coef", ascending=False))

coef_df.head(20)

In [ ]:
coef_df.tail(20)

Наибольшее влияние на вероятность оттока оказывают признаки, связанные с качеством работы приложения (количество падений) и активностью пользователя.<br>
Добавленный нелинейный признак sqrt_app_opens_per_week вошёл в число наиболее значимых, что подтверждает полезность feature engineering.<br>
Часть признаков продемонстрировала близкие к нулю коэффициенты, что указывает на их слабый вклад в модель.

In [ ]:
weak_features = coef_df[coef_df["abs_coef"] < 0.02]
print("Количество слабых признаков:", len(weak_features))
weak_features.head()

Анализ коэффициентов логистической регрессии показал, что наибольшее влияние на вероятность оттока оказывают признаки, связанные с качеством работы приложения (количество падений) и активностью пользователя. Добавленный нелинейный признак sqrt_app_opens_per_week вошёл в число наиболее значимых, что подтверждает полезность feature engineering.

Некоторые признаки продемонстрировали коэффициенты, близкие к нулю (например, avg_order_value и log_total_spent_last_month). Признак avg_order_value был удалён из модели, при этом значение PR-AUC не изменилось, что подтверждает его слабый вклад в предсказание.

Таким образом, модель была упрощена без потери качества.

## Этап 6. Эксперименты с гиперпараметрами

1. Перечислите все гиперпараметры, с которыми планируете экспериментировать.

2. Проведите систематический перебор гиперпараметров для `LogisticRegression`, выполните кросс-валидацию для каждой конфигурации.

3. Составьте таблицу с результатами.

4. Выберите лучшую модель, ориентируясь на заданную метрику качества.

Планируется экспериментировать с гиперпараметрами LogisticRegression: C (0.01, 0.1, 1, 5, 10), penalty (l1, l2), solver (liblinear, lbfgs) и class_weight (None, balanced).

In [ ]:
pipe_fe.set_params(model=LogisticRegression(max_iter=1000, random_state=42))

param_grid = [
    {
        "model__penalty": ["l2"],
        "model__solver": ["lbfgs"],
        "model__C": [0.01, 0.1, 1, 5, 10],
        "model__class_weight": [None, "balanced"],
    },
    {
        "model__penalty": ["l1"],
        "model__solver": ["liblinear"],
        "model__C": [0.01, 0.1, 1, 5, 10],
        "model__class_weight": [None, "balanced"],
    },
    {
        "model__penalty": ["l2"],
        "model__solver": ["liblinear"],
        "model__C": [0.01, 0.1, 1, 5, 10],
        "model__class_weight": [None, "balanced"],
    },
]

grid = GridSearchCV(
    estimator=pipe_fe,
    param_grid=param_grid,
    scoring="average_precision",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=False
)

grid.fit(X_reduced, y)

print("Best PR-AUC:", grid.best_score_)
print("Best params:", grid.best_params_)

В результате систематического перебора гиперпараметров LogisticRegression была получена лучшая конфигурация с L1-регуляризацией (penalty='l1'), C=0.1 и solver='liblinear'.

Лучшее значение PR-AUC составило 0.726, что превосходит результаты базовой модели и модели с добавленными признаками без настройки гиперпараметров.

Использование L1-регуляризации позволило автоматически выполнить дополнительный отбор признаков и повысить устойчивость модели.

## Этап 7. Подготовка финальной модели

Объедините лучшую конфигурацию гиперпараметров с оптимальным набором признаков. Обучите модель на всех данных для кросс-валидации и проведите финальную оценку на отложенной тестовой выборке.


In [ ]:
final_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    penalty="l1",
    solver="liblinear",
    C=0.1,
    class_weight=None
)

pipe_final = Pipeline([
    ("feature_engineering", feature_engineering),
    ("preprocessing", preprocessor_fe),
    ("model", final_model)
])

In [ ]:
X_train_red = X_train.drop(columns=["avg_order_value"])
X_test_red  = X_test.drop(columns=["avg_order_value"])

In [ ]:
pipe_final.fit(X_train_red, y_train)

y_proba_test = pipe_final.predict_proba(X_test_red)[:, 1]
test_pr_auc = average_precision_score(y_test, y_proba_test)

print("Final TEST PR-AUC:", test_pr_auc)

Лучшая конфигурация LogisticRegression (penalty='l1', solver='liblinear', C=0.1) была объединена с оптимальным набором признаков и блоком feature engineering.

Модель обучена на всей обучающей выборке и протестирована на отложенной тестовой выборке.

Итоговое значение метрики PR-AUC на тестовой выборке составило 0.81, что превышает результаты базовой модели и промежуточных конфигураций.

Полученный результат свидетельствует о хорошей способности модели выявлять клиентов, склонных к оттоку, при наличии дисбаланса классов

## Этап 8. Отчёт о проделанной работе

Проанализируйте итоговые метрики модели и факторы, которые на них повлияли. Составьте описание, выделив наиболее важные факторы.

В ходе проекта была построена модель бинарной классификации для прогнозирования оттока клиентов с использованием метрики Precision-Recall AUC, ориентированной на корректное выявление редкого класса (уходящих клиентов).

Динамика качества модели

- Базовая модель (LogisticRegression без дополнительных признаков): PR-AUC = 0.656

- После feature engineering: PR-AUC = 0.691

- После подбора гиперпараметров: PR-AUC = 0.726

- Финальная модель на тестовой выборке: PR-AUC = 0.81

Качество модели последовательно улучшалось на каждом этапе разработки.

Факторы, повлиявшие на рост качества
Добавление нелинейных признаков

В рамках feature engineering были добавлены:

- логарифмические преобразования финансовых показателей,

- квадрат частоты заказов,

- квадратный корень активности в приложении.

Наиболее значимым оказался признак sqrt_app_opens_per_week, что указывает на нелинейную зависимость между активностью пользователя и вероятностью оттока. Это подтверждает важность преобразований признаков для линейных моделей.

Регуляризация и подбор гиперпараметров

Наилучший результат показала модель с L1-регуляризацией (C=0.1).
L1-регуляризация позволила:

- автоматически занулить часть менее значимых коэффициентов,

- снизить переобучение,

- повысить обобщающую способность модели.

Упрощение набора признаков

Анализ коэффициентов позволил выявить признаки с минимальным вкладом в модель.
Удаление одного из таких признаков не ухудшило качество, что позволило упростить модель без потери эффективности.

Наиболее важные признаки

Согласно величине коэффициентов, наибольшее влияние на вероятность оттока оказывают:

- количество сбоев приложения (app_crashes_last_month),

- активность пользователя в приложении,

- некоторые поведенческие и региональные признаки.

Это указывает на то, что техническое качество сервиса и вовлечённость пользователя являются ключевыми факторами удержания клиентов.

Финальная модель демонстрирует устойчивое качество на тестовой выборке (PR-AUC = 0.81) и способна эффективно выделять клиентов с высоким риском оттока.

Модель интерпретируема, устойчива к переобучению и готова к использованию в бизнес-процессах, связанных с удержанием клиентов.

## Этап 9. Сохранение модели для продакшена

Сохраните итоговую модель и пайплайн предобработки. Убедитесь, что всё работает: загрузите артефакты и проверьте их на тестовых данных. В решении укажите ссылку для скачивания сохранённых файлов.

In [ ]:
joblib.dump(pipe_final, "coffee_churn_model.joblib")

loaded_model = joblib.load("coffee_churn_model.joblib")

y_proba_loaded = loaded_model.predict_proba(X_test_red)[:, 1]
test_pr_auc_loaded = average_precision_score(y_test, y_proba_loaded)

print("Loaded model TEST PR-AUC:", test_pr_auc_loaded)

Финальный пайплайн, включающий feature engineering, предобработку данных и обученную модель LogisticRegression, был сохранён с использованием библиотеки joblib.